<a href="https://colab.research.google.com/github/martindevoto/machine-learning-notebooks-personal/blob/main/Intro_Haystack_pt_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open-Domain QA on Tables
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial15_TableQA.ipynb)

This tutorial shows you how to perform question-answering on tables using the `TableTextRetriever` or `ElasticsearchRetriever` as retriever node and the `TableReader` as reader node.

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Thu Feb 10 18:36:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# The TaPAs-based TableReader requires the torch-scatter library
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-qxcu_4ij/farm-haystack_8c185c6927504ff9a763a18520b0fda7
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-qxcu_4ij/farm-haystack_8c185c6927504ff9a763a18520b0fda7
  Resolved https://github.com/deepset-ai/haystack.git to commit f11494a9d0c09505d149a08dfb3269145c4d8ff1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

### Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (e.g. in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [ ]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# We want to use a small model producing 512-dimensional embeddings, so we need to set embedding_dim to 512
document_index = "document"
document_store = ElasticsearchDocumentStore(
    host="localhost", username="", password="", index=document_index, embedding_dim=512
)

## Add Tables to DocumentStore
To quickly demonstrate the capabilities of the `TableTextRetriever` and the `TableReader` we use a subset of 1000 tables of the [Open Table-and-Text Question Answering (OTT-QA) dataset](https://github.com/wenhuchen/OTT-QA).

Just as text passages, tables are represented as `Document` objects in Haystack. The content field, though, is a pandas DataFrame instead of a string.

In [ ]:
# Let's first fetch some tables that we want to query
# Here: 1000 tables from OTT-QA
from haystack.utils import fetch_archive_from_http

doc_dir = "data"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/ottqa_tables_sample.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/ottqa_tables_sample.json.zip to `data`


True

In [ ]:
# Add the tables to the DocumentStore

import json
from haystack import Document
import pandas as pd

def read_ottqa_tables(filename):
  processed_tables = []
  with open(filename) as tables:
    tables = json.load(tables)
    for key, table in tables.items():
      current_columns = table['header']
      current_rows = table['data']
      current_df = pd.DataFrame(columns=current_columns, 
                                data=current_rows)
      current_doc_title = table['title']
      current_section_title = table['section_title']
      document = Document(
          content=current_df,
          content_type='table',
          meta={'title': current_doc_title,
                'section_title': current_section_title},
          id=key,
      )
      processed_tables.append(document)

  return processed_tables

tables = read_ottqa_tables(f"{doc_dir}/ottqa_tables_sample.json")
document_store.write_documents(tables, index=document_index)

# Showing content filed and meta field of one of te Documents of content_type
# 'table'

print(tables[0].content)
print(tables[0].meta)

      Result  ...            Score
0     Winner  ...        6-1 , 6-1
1     Winner  ...  6-2 , 4-6 , 6-3
2     Winner  ...        6-2 , 6-2
3  Runner-up  ...        3-6 , 2-6
4     Winner  ...  6-7 , 6-3 , 6-0
5     Winner  ...        6-1 , 6-0
6     Winner  ...  6-2 , 2-6 , 6-2
7     Winner  ...        6-0 , 6-4

[8 rows x 8 columns]
{'title': 'Rewa Hudson', 'section_title': 'ITF finals ( 7–3 ) -- Doubles ( 7–1 )'}


## Initalize Retriever, Reader, & Pipeline

### Retriever

Retrievers help narrowing down the scope for the Reader to a subset of tables where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use the `TableTextRetriever` capable of retrieving relevant content among a database
of texts and tables using dense embeddings. It is an extension of the `DensePassageRetriever` and consists of three encoders (one query encoder, one text passage encoder and one table encoder) that create embeddings in the same vector space. More details on the `TableTextRetriever` and how it is trained can be found in [this paper](https://arxiv.org/abs/2108.04049).

**Alternatives:**

- `ElasticsearchRetriever` that uses BM25 algorithm


In [ ]:
from haystack.nodes.retriever import TableTextRetriever

retriever = TableTextRetriever(
    document_store=document_store,
    query_embedding_model="deepset/bert-small-mm_retrieval-question_encoder",
    passage_embedding_model="deepset/bert-small-mm_retrieval-passage_encoder",
    table_embedding_model="deepset/bert-small-mm_retrieval-table_encoder",
    embed_meta_fields=["title", "section_title"],
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/355 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/bert-small-mm_retrieval-table_encoder locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/110M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/bert-small-mm_retrieval-table_encoder


In [ ]:
# Add table embeddings to the tables in DocumentStore
document_store.update_embeddings(retriever=retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 1000 docs ...


Updating embeddings:   0%|          | 0/1000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1008 [00:00<?, ? Docs/s]

In [ ]:
## Alternative: ElasticsearchRetriever
# from haystack.nodes.retriever import ElasticsearchRetriever
# retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# Try the Retriever
from haystack.utils import print_documents

retrieved_tables = retriever.retrieve("How many twin buildings are under construction?", top_k=5)
# Get highest scored table
print(retrieved_tables[0].content)

                             Name  ...              Status
0                  Twin Towers II  ...         Never built
1              World Trade Center  ...           Destroyed
2                Three Sixty West  ...  Under construction
3                  Gateway Towers  ...  Under construction
4                 Rustomjee Crown  ...  Under construction
5                  Orchid Heights  ...             On-hold
6                Hermitage Towers  ...            Proposed
7             Lokhandwala Minerva  ...  Under construction
8                    Lamar Towers  ...  Under construction
9            Indonesia One Towers  ...  Under construction
10                       Sky link  ...            Approved
11                  Vida Za'abeel  ...            Proposed
12  Broadway Corridor Twin Towers  ...         Never built
13   India Bulls Sky Forest Tower  ...  Under construction
14                 Capital Towers  ...  Under construction
15               One Avighna Park  ...  Under constructi

### Reader
The `TableReader` is based on TaPas, a transformer-based language model capable of grasping the two-dimensional structure of a table. It scans the tables returned by the retriever and extracts the answer. The available TableReader models can be found [here](https://huggingface.co/models?pipeline_tag=table-question-answering&sort=downloads).

**Notice**: The `TableReader` will return an answer for each table, even if the query cannot be answered by the table. Furthermore, the confidence scores are not useful as of now, given that they will *always* be very high (i.e. 1 or close to 1).

In [ ]:
from haystack.nodes import TableReader

reader = TableReader(model_name_or_path="google/tapas-base-finetuned-wtq", max_seq_len=512)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

In [ ]:
# Try the TableReader on one Table (highest-scored retrieved table from previous section)

table_doc = document_store.get_document_by_id("List_of_tallest_twin_buildings_and_structures_in_the_world_1")
print(table_doc.content)

                             Name  ...              Status
0                  Twin Towers II  ...         Never built
1              World Trade Center  ...           Destroyed
2                Three Sixty West  ...  Under construction
3                  Gateway Towers  ...  Under construction
4                 Rustomjee Crown  ...  Under construction
5                  Orchid Heights  ...             On-hold
6                Hermitage Towers  ...            Proposed
7             Lokhandwala Minerva  ...  Under construction
8                    Lamar Towers  ...  Under construction
9            Indonesia One Towers  ...  Under construction
10                       Sky link  ...            Approved
11                  Vida Za'abeel  ...            Proposed
12  Broadway Corridor Twin Towers  ...         Never built
13   India Bulls Sky Forest Tower  ...  Under construction
14                 Capital Towers  ...  Under construction
15               One Avighna Park  ...  Under constructi

In [ ]:
from haystack.utils import print_answers

prediction = reader.predict(query="How many twin buildings are under construction?", documents=[table_doc])
print_answers(prediction, details="all")


Query: How many twin buildings are under construction?
Answers:
[   <Answer {'answer': '12', 'type': 'extractive', 'score': 1.0, 'context':                              Name  ...              Status
0                  Twin Towers II  ...         Never built
1              World Trade Center  ...           Destroyed
2                Three Sixty West  ...  Under construction
3                  Gateway Towers  ...  Under construction
4                 Rustomjee Crown  ...  Under construction
5                  Orchid Heights  ...             On-hold
6                Hermitage Towers  ...            Proposed
7             Lokhandwala Minerva  ...  Under construction
8                    Lamar Towers  ...  Under construction
9            Indonesia One Towers  ...  Under construction
10                       Sky link  ...            Approved
11                  Vida Za'abeel  ...            Proposed
12  Broadway Corridor Twin Towers  ...         Never built
13   India Bulls Sky Forest Tower

The offsets in the `offsets_in_document` and `offsets_in_context` field indicate the table cells that the model predicts to be part of the answer. They need to be interpreted on the linearized table, i.e., a flat list containing all of the table cells.

In the `Answer`'s meta field, you can find the aggreagtion operator used to construct the answer (in this case `COUNT`) and the answer cells as strings.

In [ ]:
print(f"Predicted answer: {prediction['answers'][0].answer}")
print(f"Meta field: {prediction['answers'][0].meta}")

Predicted answer: 12
Meta field: {'aggregation_operator': 'COUNT', 'answer_cells': ['Three Sixty West', 'Gateway Towers', 'Rustomjee Crown', 'Lokhandwala Minerva', 'Lamar Towers', 'Indonesia One Towers', 'India Bulls Sky Forest Tower', 'Capital Towers', 'One Avighna Park', 'The Destiny ( Tower )', 'Oberoi Esquire Towers', 'Bhoomi Celestia']}


### Pipeline
The Retriever and the Reader can be sticked together to a pipeline in order to first retrieve relevant tables and then extract the answer.

**Notice**: Given that the `TableReader` does not provide useful confidence scores and returns an answer for each of the tables, the sorting of the answers might be not helpful.

In [ ]:
# Initialize pipeline
from haystack import Pipeline

table_qa_pipeline = Pipeline()
table_qa_pipeline.add_node(component=retriever, name="TableTextRetriever", inputs=["Query"])
table_qa_pipeline.add_node(component=reader, name="TableReader", inputs=["TableTextRetriever"])

In [ ]:
prediction = table_qa_pipeline.run("How many twin buildings are under construction?")
print_answers(prediction, details="minimum")


Query: How many twin buildings are under construction?
Answers:
[   {   'answer': '12',
        'context':                              Name  ...              Status
0                  Twin Towers II  ...         Never built
1              World Trade Center  ...           Destroyed
2                Three Sixty West  ...  Under construction
3                  Gateway Towers  ...  Under construction
4                 Rustomjee Crown  ...  Under construction
5                  Orchid Heights  ...             On-hold
6                Hermitage Towers  ...            Proposed
7             Lokhandwala Minerva  ...  Under construction
8                    Lamar Towers  ...  Under construction
9            Indonesia One Towers  ...  Under construction
10                       Sky link  ...            Approved
11                  Vida Za'abeel  ...            Proposed
12  Broadway Corridor Twin Towers  ...         Never built
13   India Bulls Sky Forest Tower  ...  Under construction
14     